# **Object Detection - Models Training**

---

In [1]:
from collections import Counter
from creds import RF_API_KEY, RF_WORKSPACE_ID, RF_PROJECT_ID
from glob import glob
import os
import random
from rfdetr import RFDETRBase
from roboflow import Roboflow
import shutil
import torch
from ultralytics import YOLO

In [2]:
def get_image_label_pairs(
    image_root, annotation_root, image_ext="jpg", label_ext="txt"
):
    pairs = []
    image_files = glob(os.path.join(image_root, "**", f"*.{image_ext}"), recursive=True)

    for img in image_files:
        rel_path = os.path.relpath(img, image_root)
        label = os.path.join(
            annotation_root, os.path.splitext(rel_path)[0] + f".{label_ext}"
        )

        if os.path.exists(label):
            pairs.append((img, label))
        else:
            print(f"Warning: Label file not found for image {img}")

    return pairs

In [3]:
image_root = "../../data/augmented_data/frames"
annotation_root = "../../data/annotations"
pairs = get_image_label_pairs(image_root, annotation_root)
print(f"Found {len(pairs)} image-label pairs.")

Found 19236 image-label pairs.


In [4]:
empty_files = []

for root, _, files in os.walk(annotation_root):
    for file in files:
        file_path = os.path.join(root, file)

        if os.stat(file_path).st_size == 0:
            empty_files.append(file_path)

print(f"Found {len(empty_files)} empty label files.")

Found 0 empty label files.


In [5]:
label_counts = Counter()

for root, _, files in os.walk(annotation_root):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            lines = f.readlines()

            for line in lines:
                class_id = line.split()[0]
                label_counts[class_id] += 1

label_counts

Counter({'0': 31663, '1': 30627, '4': 30587, '2': 30072, '3': 27189})

In [6]:
# Paths
image_dir = "../../data/augmented_data/frames"
label_dir = "../../data/annotations"
output_dir = "../../data/split_data"

# Create output folders
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

# Get all images
all_images = []
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            all_images.append(os.path.join(root, file))

# Shuffle and split
random.shuffle(all_images)
train_ratio = 0.7
val_ratio = 0.2
train_count = int(len(all_images) * train_ratio)
val_count = int(len(all_images) * val_ratio)

train_images = all_images[:train_count]
val_images = all_images[train_count : train_count + val_count]
test_images = all_images[train_count + val_count :]

# Move files
def move_files(image_list, split):
    for img_path in image_list:
        # Get label path
        rel_path = os.path.relpath(img_path, image_dir)
        label_path = os.path.join(label_dir, os.path.splitext(rel_path)[0] + ".txt")

        # Define new paths
        new_img_path = os.path.join(
            output_dir, "images", split, os.path.basename(img_path)
        )
        new_label_path = os.path.join(
            output_dir, "labels", split, os.path.basename(label_path)
        )

        # Copy files
        shutil.copy(img_path, new_img_path)
        if os.path.exists(label_path):
            shutil.copy(label_path, new_label_path)


# Move train, val & test files
move_files(train_images, "train")
move_files(val_images, "val")
move_files(test_images, "test")

print(
    f"Dataset split completed: {len(train_images)} train, {len(val_images)} val, {len(test_images)} test"
)

Dataset split completed: 13465 train, 3847 val, 1924 test


In [7]:
yolo = YOLO(verbose=False)

results = yolo.train(
    data="../../data/dataset.yaml",
    epochs=100,
    batch=4,
    device="cuda" if torch.cuda.is_available() else "cpu",
    multi_scale=True,
    patience=5,
    lr0=1e-4,
    augment=True,
)

100%|██████████| 5.35M/5.35M [00:00<00:00, 9.59MB/s]


New https://pypi.org/project/ultralytics/8.3.143 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.133  Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../data/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=True, name=train, nbs=64, nms=False, opset=None,

train: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\train... 2236 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2236/2236 [00:05<00:00, 375.19it/s]

train: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\train.cache


val: Fast image access  (ping: 0.10.0 ms, read: 9.27.1 MB/s, size: 119.7 KB)


...

     46/100      2.19G     0.7773     0.4191     0.8132         30        736: 100%|██████████| 559/559 [01:06<00:00,  8.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 239/239 [00:18<00:00, 12.67it/s]

                   all       1905      15238      0.988      0.975      0.991      0.864
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 41, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



46 epochs completed in 1.105 hours.
Optimizer stripped from C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train\weights\best.pt, 5.5MB

Validating C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train\weights\best.pt...
Ultralytics 8.3.133  Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 239/239 [00:26<00:00,  8.96it/s]


                   all       1905      15238      0.987      0.966       0.99       0.79
                  blue       1644       3478       0.99      0.965      0.984       0.79
                 green       1610       3106       0.99      0.966      0.993      0.797
                orange       1577       2991      0.984      0.984      0.994      0.807
                   red       1341       2646      0.982      0.959      0.985      0.783
                yellow       1529       3017      0.989      0.956      0.992      0.774
Speed: 0.2ms preprocess, 6.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train


In [2]:
rf = Roboflow(api_key=RF_API_KEY)
project = rf.workspace(RF_WORKSPACE_ID).project(RF_PROJECT_ID)
version = project.version(3)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Seminar-ObjectDetection-3 in coco:: 100%|██████████| 13071/13071 [00:11<00:00, 1144.67it/s]


In [2]:
rfdetr = RFDETRBase()

rfdetr.train(
    dataset_dir="../../data/coco_json",
    epochs=100,
    batch_size=4,
    grad_accum_steps=4,
    lr=1e-4,
    seed=124256,
    early_stopping=True,
    early_stopping_patience=5,
    tensorboard=False,
    resume="output/checkpoint.pth",
)

Loading pretrain weights


num_classes mismatch: model has 90 classes, but your dataset has 6 classes
reinitializing your detection head with 6 classes.


Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=6, grad_accum_steps=4, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=4, weight_decay=0.0001, epochs=100, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-base.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[2, 5, 8, 11], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=3, dim_feedforward=2048, hidden_dim=256, sa_nheads=8, ca_nheads=16, num_queries=300, group_detr=13, two_stage=True, projector_scale=['P4'], lite_refpoint_refine=True, num_select=300, dec_n_poi

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)


Epoch: [29]  [  0/714]  eta: 8:48:07  lr: 0.000100  class_error: 0.00  loss: 1.5783 (1.5783)  loss_ce: 0.2370 (0.2370)  loss_bbox: 0.0127 (0.0127)  loss_giou: 0.1309 (0.1309)  loss_ce_0: 0.2449 (0.2449)  loss_bbox_0: 0.0127 (0.0127)  loss_giou_0: 0.1292 (0.1292)  loss_ce_1: 0.2376 (0.2376)  loss_bbox_1: 0.0128 (0.0128)  loss_giou_1: 0.1312 (0.1312)  loss_ce_enc: 0.2658 (0.2658)  loss_bbox_enc: 0.0137 (0.0137)  loss_giou_enc: 0.1498 (0.1498)  loss_ce_unscaled: 0.2370 (0.2370)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.0025 (0.0025)  loss_giou_unscaled: 0.0654 (0.0654)  cardinality_error_unscaled: 2714.0000 (2714.0000)  loss_ce_0_unscaled: 0.2449 (0.2449)  loss_bbox_0_unscaled: 0.0025 (0.0025)  loss_giou_0_unscaled: 0.0646 (0.0646)  cardinality_error_0_unscaled: 1179.0000 (1179.0000)  loss_ce_1_unscaled: 0.2376 (0.2376)  loss_bbox_1_unscaled: 0.0026 (0.0026)  loss_giou_1_unscaled: 0.0656 (0.0656)  cardinality_error_1_unscaled: 1458.0000 (1458.0000)  loss_ce_enc_unscale